<a href="https://colab.research.google.com/github/farroshsy/MidtermExam_BigData/blob/main/SparkStreaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Midterm Exam**
## Spark Streaming and Reccomendation Systems
---

Name: Farros Hilmi Syafei 
<br>
Student ID: 5025201012
<br>
Class: Big Data A
<br>
Lecturer: Abdul Munif, S.Kom., M.Sc.


## Problem
### 1. Spark Streaming
Create an Apache Spark notebook for handling file stream inside a folder.
*   Use the data given in news folder (news-1.json, news-2.json, news-3.json)
*   Put the input inside folder named input-your-student-id. Example: input-51231132
*   Put the output inside folder named output-your-student-id




## Initialization

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=f772e0f8377c7efc3117cba2e63ed1257d5b99dec6cc5c0fc8fd3fd07ddbea62
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


## Initiate the Spark session

In [52]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.streaming import StreamingContext
import os

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkStreaming") \
    .getOrCreate()

In [53]:
# Check spark session
print(spark)

In [54]:
# Set the log level to ERROR
spark.sparkContext.setLogLevel("ERROR")

In [55]:
import os

# Set the input and output folder paths
input_dir = "./content/input-5025201012"
output_dir = "/content/output-5025201012"

if not os.path.exists(input_dir):
    os.makedirs(input_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Define the data structure

In [56]:

# Define the schema
schema = StructType([
    StructField("link", StringType(), True),
    StructField("headline", StringType(), True),
    StructField("category", StringType(), True),
    StructField("short_description", StringType(), True),
    StructField("authors", StringType(), True),
    StructField("date", StringType(), True),
])

In [64]:
# Create a StreamingContext with batch interval of 5 seconds
ssc = StreamingContext(spark.sparkContext, 5)

In [58]:
# Create the streaming DataFrame to read from the input directory
streaming_df = spark.readStream \
    .format("json") \
    .schema(schema) \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "data/archive/device_data/") \
    .option("maxFilesPerTrigger", 1) \
    .load(input_dir)

## define a function to process each file

## Determine the folder



In [60]:
df.printSchema()

root
 |-- link: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- category: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- date: string (nullable = true)



In [61]:
def process_file(spark, file_path):
    # Read the JSON file and count the number of lines
    df = spark.read.json(file_path)
    count = df.count()

    # Write the result to a file in the output directory
    output_file_path = file_path.replace('input', 'output')
    df.write.json(output_file_path)

In [62]:
# Create a StreamingContext with batch interval of 5 seconds
ssc = StreamingContext(spark.sparkContext, 5)

# Monitor the input directory for new files
input_stream = ssc.textFileStream(input_dir)

# Process each file as it appears in the directory
input_stream.foreachRDD(lambda rdd: rdd.foreach(lambda file_path: process_file(spark, file_path)))


In [63]:


# Start the new StreamingContext
ssc.start()
ssc.awaitTermination()

Py4JJavaError: ignored

In [ ]:
 # Read JSON files from the input folder
news_df = spark.readStream.schema(schema).json(input_dir)

In [ ]:
# Group by category and count the number of news items in each category
grouped_news_df = news_df.groupBy("category").count()
from pyspark.sql.functions import from_unixtime, unix_timestamp, window

news_df_with_timestamp = news_df.withColumn("timestamp", unix_timestamp("date", "yyyy-MM-dd").cast("timestamp"))

watermarked_news_df = news_df_with_timestamp.withWatermark("timestamp", "1 minutes")

grouped_news_df = watermarked_news_df.groupBy(window("timestamp", "1 minutes"), "category").count()


## Start the streaming task

In [ ]:
result = groupDF.writeStream\
  .format("console")\
  .outputMode("complete")\
  .start()


result.awaitTermination()